In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
import cudf as pd
import cupy as np
import cuml

In [5]:
train_df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test_df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [6]:
train_df.head()
test_df.head()

In [7]:
# Checking the number columns

len(train_df.columns)

In [8]:
train_df.info()

In [9]:
test_df.info()

In [10]:
train_df.shape

In [11]:
Y = train_df['SalePrice']
train_df.drop('SalePrice',axis=1,inplace=True)

In [14]:
#Missing values
train_df.isnull().sum()
test_df.isnull().sum()

In [19]:
train_df['LotFrontage']=train_df['LotFrontage'].fillna(train_df['LotFrontage'].mean())
train_df['LotFrontage'].value_counts()
train_df['BsmtCond']=train_df['BsmtCond'].fillna(train_df['BsmtCond'].mode()[0])
train_df['BsmtQual']=train_df['BsmtQual'].fillna(train_df['BsmtQual'].mode()[0])
train_df['FireplaceQu']=train_df['FireplaceQu'].fillna(train_df['FireplaceQu'].mode()[0])
train_df['GarageType']=train_df['GarageType'].fillna(train_df['GarageType'].mode()[0])
train_df['GarageFinish']=train_df['GarageFinish'].fillna(train_df['GarageFinish'].mode()[0])
train_df['GarageQual']=train_df['GarageQual'].fillna(train_df['GarageQual'].mode()[0])
train_df['GarageCond']=train_df['GarageCond'].fillna(train_df['GarageCond'].mode()[0])
train_df['MasVnrType']=train_df['MasVnrType'].fillna(train_df['MasVnrType'].mode()[0])
train_df['MasVnrArea']=train_df['MasVnrArea'].fillna(train_df['MasVnrArea'].mode()[0])
train_df['BsmtExposure']=train_df['BsmtExposure'].fillna(train_df['BsmtExposure'].mode()[0])
train_df['BsmtFinType2']=train_df['BsmtFinType2'].fillna(train_df['BsmtFinType2'].mode()[0])
train_df['Utilities']=train_df['Utilities'].fillna(train_df['Utilities'].mode()[0])
train_df['Exterior1st']=train_df['Exterior1st'].fillna(train_df['Exterior1st'].mode()[0])
train_df['Exterior2nd']=train_df['Exterior2nd'].fillna(train_df['Exterior2nd'].mode()[0])
train_df['BsmtFinType1']=train_df['BsmtFinType1'].fillna(train_df['BsmtFinType1'].mode()[0])
train_df['SaleType']=train_df['SaleType'].fillna(train_df['SaleType'].mode()[0])
train_df['BsmtFullBath']=train_df['BsmtFullBath'].fillna(train_df['BsmtFullBath'].mode()[0])
train_df['BsmtHalfBath']=train_df['BsmtHalfBath'].fillna(train_df['BsmtHalfBath'].mode()[0])
train_df['KitchenQual']=train_df['KitchenQual'].fillna(train_df['KitchenQual'].mode()[0])
train_df['Functional']=train_df['Functional'].fillna(train_df['Functional'].mode()[0])
train_df['GarageCars']=train_df['GarageCars'].fillna(train_df['GarageCars'].mean())
train_df['GarageArea']=train_df['GarageArea'].fillna(train_df['GarageArea'].mean())
train_df['BsmtFinSF1']=train_df['BsmtFinSF1'].fillna(train_df['BsmtFinSF1'].mean())
train_df['BsmtFinSF2']=train_df['BsmtFinSF2'].fillna(train_df['BsmtFinSF2'].mean())
train_df['BsmtUnfSF']=train_df['BsmtUnfSF'].fillna(train_df['BsmtUnfSF'].mean())
train_df['TotalBsmtSF']=train_df['TotalBsmtSF'].fillna(train_df['TotalBsmtSF'].mean())


#train_df['MasVnrArea'].fillna(train_df['MasVnrArea'].median(), inplace=True)
#train_df['BsmtFinType1'].fillna('Unknown', inplace=True)
#train_df['BsmtFinType2'].fillna('Unknown', inplace=True)

In [21]:
test_df['LotFrontage']=test_df['LotFrontage'].fillna(test_df['LotFrontage'].mean())
test_df['LotFrontage'].value_counts()
test_df['BsmtCond']=test_df['BsmtCond'].fillna(test_df['BsmtCond'].mode()[0])
test_df['BsmtQual']=test_df['BsmtQual'].fillna(test_df['BsmtQual'].mode()[0])
test_df['FireplaceQu']=test_df['FireplaceQu'].fillna(test_df['FireplaceQu'].mode()[0])
test_df['GarageType']=test_df['GarageType'].fillna(test_df['GarageType'].mode()[0])
test_df['GarageFinish']=test_df['GarageFinish'].fillna(test_df['GarageFinish'].mode()[0])
test_df['GarageQual']=test_df['GarageQual'].fillna(test_df['GarageQual'].mode()[0])
test_df['GarageCond']=test_df['GarageCond'].fillna(test_df['GarageCond'].mode()[0])
test_df['MasVnrType']=test_df['MasVnrType'].fillna(test_df['MasVnrType'].mode()[0])
test_df['MasVnrArea']=test_df['MasVnrArea'].fillna(test_df['MasVnrArea'].mode()[0])
test_df['BsmtExposure']=test_df['BsmtExposure'].fillna(test_df['BsmtExposure'].mode()[0])
test_df['BsmtFinType2']=test_df['BsmtFinType2'].fillna(test_df['BsmtFinType2'].mode()[0])
test_df['MSZoning']=test_df['MSZoning'].fillna(test_df['MSZoning'].mode()[0])
test_df['Utilities']=test_df['Utilities'].fillna(test_df['Utilities'].mode()[0])
test_df['Exterior1st']=test_df['Exterior1st'].fillna(test_df['Exterior1st'].mode()[0])
test_df['Exterior2nd']=test_df['Exterior2nd'].fillna(test_df['Exterior2nd'].mode()[0])
test_df['BsmtFinType1']=test_df['BsmtFinType1'].fillna(test_df['BsmtFinType1'].mode()[0])
test_df['SaleType']=test_df['SaleType'].fillna(test_df['SaleType'].mode()[0])
test_df['BsmtFullBath']=test_df['BsmtFullBath'].fillna(test_df['BsmtFullBath'].mode()[0])
test_df['BsmtHalfBath']=test_df['BsmtHalfBath'].fillna(test_df['BsmtHalfBath'].mode()[0])
test_df['KitchenQual']=test_df['KitchenQual'].fillna(test_df['KitchenQual'].mode()[0])
test_df['Functional']=test_df['Functional'].fillna(test_df['Functional'].mode()[0])
test_df['GarageCars']=test_df['GarageCars'].fillna(test_df['GarageCars'].mean())
test_df['GarageArea']=test_df['GarageArea'].fillna(test_df['GarageArea'].mean())
test_df['BsmtFinSF1']=test_df['BsmtFinSF1'].fillna(test_df['BsmtFinSF1'].mean())
test_df['BsmtFinSF2']=test_df['BsmtFinSF2'].fillna(test_df['BsmtFinSF2'].mean())
test_df['BsmtUnfSF']=test_df['BsmtUnfSF'].fillna(test_df['BsmtUnfSF'].mean())
test_df['TotalBsmtSF']=test_df['TotalBsmtSF'].fillna(test_df['TotalBsmtSF'].mean())

In [22]:
train_df.drop(['Alley','PoolQC','Fence','MiscFeature','GarageYrBlt','Id'],axis=1,inplace=True)
test_df.drop(['Alley','PoolQC','Fence','MiscFeature','GarageYrBlt','Id'],axis=1,inplace=True)

In [24]:
train_df.isnull().sum()

In [25]:
train_df=train_df.fillna(method ='pad')

In [27]:
df=pd.concat([train_df,test_df],axis=0)
df=pd.get_dummies(df)

In [28]:
df.head()

In [30]:
#Linear Regression
num_train,num_test = train_df.shape[0],test_df.shape[0]
train_df = df.iloc[:num_train,:]
test_df = df.iloc[num_test+1:,:]

In [32]:
X=train_df

In [33]:
X.shape

In [35]:
from cuml.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, Y,test_size=0.29,random_state=42)

In [36]:
from cuml import LinearRegression
lr = LinearRegression(fit_intercept = True, normalize = False, algorithm = 'svd-jacobi')
split=lr.fit(X,Y)

In [37]:
print("R-Squared Value for Training Set: {:.3f}".format(lr.score(X_train.astype('float32'), y_train.astype('float32'))))

In [39]:
y_pred_split=lr.predict(test_df)
y_pred_split

In [ ]:
#Evaluation Metrics for svd

In [40]:
print("Algorithm: svd")
lr = LinearRegression(fit_intercept = True, normalize = False, algorithm = 'svd')
reg = lr.fit(X_train,y_train)
prediction = lr.predict(X_val)
print("MSE:")
print(cuml.metrics.regression.mean_squared_error(y_val.astype('int64'),prediction.astype('int64')))
print("R2 Score:")
print(cuml.metrics.regression.r2_score(y_val.astype('float32'),prediction.astype('float32')))
print("MAE:")
print(cuml.metrics.regression.mean_absolute_error(y_val.astype('int64'),prediction.astype('int64')))

In [ ]:
# Evaluation Metrics for eig

In [42]:
print("Algorithm: eig")
lr = LinearRegression(fit_intercept = True, normalize = False, algorithm = 'eig')
reg = lr.fit(X_train,y_train)
prediction = lr.predict(X_val)
print("MSE:")
print(cuml.metrics.regression.mean_squared_error(y_val.astype('int64'),prediction.astype('int64')))
print("R2 Score:")
print(cuml.metrics.regression.r2_score(y_val.astype('float32'),prediction.astype('float32')))
print("MAE:")
print(cuml.metrics.regression.mean_absolute_error(y_val.astype('int64'),prediction.astype('int64')))

In [ ]:
# Evaluation Metrics for svd-qr

In [43]:
print("Algorithm: svd-qr")
lr = LinearRegression(fit_intercept = True, normalize = False, algorithm = 'svd-qr')
reg = lr.fit(X_train,y_train)
prediction = lr.predict(X_val)
print("MSE:")
print(cuml.metrics.regression.mean_squared_error(y_val.astype('int64'),prediction.astype('int64')))
print("R2 Score:")
print(cuml.metrics.regression.r2_score(y_val.astype('float32'),prediction.astype('float32')))
print("MAE:")
print(cuml.metrics.regression.mean_absolute_error(y_val.astype('int64'),prediction.astype('int64')))

In [ ]:
# Evaluation Metrics for svd-jacobi

In [44]:
print("Algorithm: svd-jacobi")
lr = LinearRegression(fit_intercept = True, normalize = False, algorithm = 'svd-jacobi')
reg = lr.fit(X_train,y_train)
prediction = lr.predict(X_val)
print("MSE:")
print(cuml.metrics.regression.mean_squared_error(y_val.astype('int64'),prediction.astype('int64')))
print("R2 Score:")
print(cuml.metrics.regression.r2_score(y_val.astype('float32'),prediction.astype('float32')))
print("MAE:")
print(cuml.metrics.regression.mean_absolute_error(y_val.astype('int64'),prediction.astype('int64')))

In [ ]:
# Evaluation Metrics for qr

In [45]:
print("Algorithm: qr")
lr = LinearRegression(fit_intercept = True, normalize = False, algorithm = 'qr')
reg = lr.fit(X_train,y_train)
prediction = lr.predict(X_val)
print("MSE:")
print(cuml.metrics.regression.mean_squared_error(y_val.astype('int64'),prediction.astype('int64')))
print("R2 Score:")
print(cuml.metrics.regression.r2_score(y_val.astype('float32'),prediction.astype('float32')))
print("MAE:")
print(cuml.metrics.regression.mean_absolute_error(y_val.astype('int64'),prediction.astype('int64')))

In [46]:
sample_sub = pd.read_csv('../input/house-prices-advanced-regression-techniques/sample_submission.csv')
sample_sub.head()

In [47]:
sample_sub['SalePrice']

In [ ]:
#Ridge Regression

In [49]:
from cuml import Ridge
alpha = np.array([1e-5])
r = Ridge(alpha = alpha, fit_intercept = True, normalize = False, solver = "eig")
model1=r.fit(X,Y)
y_pred_ridge = model1.predict(test_df)
y_pred_ridge

In [50]:
alpha = np.array([1e-5])
r2 = Ridge(alpha = alpha, fit_intercept = True, normalize = False, solver = "svd")
model2 = r2.fit(X,Y)
y_pred_ridge2 = model2.predict(test_df)
y_pred_ridge2

In [51]:
alpha = np.array([1e-5])
r3 = Ridge(alpha = alpha, fit_intercept = True, normalize = False, solver = "cvd")
model3 = r3.fit(X,Y)
y_pred_ridge3 = model3.predict(test_df)
y_pred_ridge3

In [ ]:
#Lasso Regression

In [52]:
from cuml.linear_model import Lasso
ls = Lasso(alpha = 0.1)
result = ls.fit(X, Y)
y_pred_lasso=result.predict(test_df)
y_pred_lasso